# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [1]:
import os
import datetime
import numpy as np
import pandas as pd

In [2]:
current_dir = %pwd
timestamp = os.path.basename(current_dir)

In [3]:
import sys
sys.path.append('../../')
%cd '../../'

/home/tsu-nera/repo/bakuchi


In [4]:
from src.utils.trade_analysis import TradeAnalysis

%reload_ext autoreload
%autoreload 2

File doesn't exist 


In [5]:
ta = TradeAnalysis(timestamp)

In [6]:
cc_trades = ta.get_coincheck_trades_df()
lq_trades = ta.get_liquid_trades_df()

In [7]:
len(cc_trades), len(lq_trades)

(246, 246)

In [8]:
cc_ticks = ta.get_coincheck_ticks_df()
lq_ticks = ta.get_liquid_ticks_df()

In [9]:
cc_ticks.head(5)

,bid,ask,tick_timestamp
timestamp,,,
2020-03-25 16:47:48,743303.0,743800.0,2020-03-25 16:47:47
2020-03-25 16:47:49,743303.0,743800.0,2020-03-25 16:47:49
2020-03-25 16:47:50,743303.0,743800.0,2020-03-25 16:47:50
2020-03-25 16:47:51,743303.0,743800.0,2020-03-25 16:47:51
2020-03-25 16:47:52,743303.0,743800.0,2020-03-25 16:47:52


In [10]:
trades = []

for cc_row, lq_row in zip(cc_trades.iterrows(), lq_trades.iterrows()):
    cc_item = cc_row[1]
    lq_item = lq_row[1]
    
    cc_side = cc_item["side"]
    cc_timestamp = cc_item["datetime"]
    cc_amount = cc_item["amount"]
    cc_actual_price = cc_item["price"]
    cc_actual_rate = cc_item["rate"]

    lq_side = lq_item["side"]
    lq_timestamp = lq_item["datetime"]
    lq_amount = lq_item["amount"]
    lq_actual_price = lq_item["price"]
    lq_actual_rate = lq_item["rate"]
    
    try:
        cc_tick = cc_ticks.loc[cc_timestamp]
    except Exception:
        timestamp = cc_timestamp + datetime.timedelta(seconds=1)
        try:
            cc_tick = cc_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            cc_tick = cc_ticks.loc[timestamp2]

    try:
        lq_tick = lq_ticks.loc[lq_timestamp]
    except Exception:
        timestamp = lq_timestamp + datetime.timedelta(seconds=1)
        try:
            lq_tick = lq_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            try:
                lq_tick = lq_ticks.loc[timestamp2]
            except Exception:
                timestamp3 = timestamp2 + datetime.timedelta(seconds=1)
                lq_tick = lq_ticks.loc[timestamp3]


    if cc_side == "sell":
        cc_tick_rate = cc_tick.bid
    else:
        cc_tick_rate = cc_tick.ask
    if lq_side == "sell":
        lq_tick_rate = lq_tick.bid
    else:
        lq_tick_rate = lq_tick.ask
        
    cc_expected_price = cc_tick_rate * cc_amount
    lq_expected_price = lq_tick_rate * lq_amount

    
    if cc_side == "buy":
        cc_expected_price = cc_expected_price * -1
        cc_actual_price = cc_actual_price * -1
    if lq_side == "buy":
        lq_expected_price = lq_expected_price * -1
        lq_actual_price = lq_actual_price * -1
        
    expected_profit = sum([cc_expected_price, lq_expected_price])
    actual_profit = sum([cc_actual_price, lq_actual_price])

    trade = {
        "timestamp": cc_timestamp, 
        "cc_side": cc_side,
        "cc_expected_rate": cc_tick_rate,
        "cc_actual_rate": cc_actual_rate,
        "cc_expected_price": cc_expected_price,
        "cc_actual_price": cc_actual_price,
        "lq_side": lq_side,
        "lq_expected_rate": lq_tick_rate,
        "lq_actual_rate": lq_actual_rate,
        "lq_expected_price": lq_expected_price,
        "lq_actual_price": lq_actual_price,
        "expected_profit": expected_profit,
        "actual_profit": actual_profit
    }
    
    trades.append(trade)

In [11]:
trade_df = pd.DataFrame(trades)
trade_df = trade_df.set_index("timestamp")

In [12]:
profit_df = trade_df[["expected_profit", "actual_profit"]]

In [13]:
int(sum(profit_df["expected_profit"])), int(sum(profit_df["actual_profit"]))

(1534, 893)

In [14]:
pd.set_option('display.max_rows', None)
profit_df

,expected_profit,actual_profit
timestamp,,
2020-03-25 17:03:03,7.470000,-0.576
2020-03-25 17:11:35,2.002760,-1.312
2020-03-25 17:58:47,11.470498,-3.041
2020-03-25 17:58:57,4.254000,-1.224
2020-03-25 17:59:14,15.496100,2.725
2020-03-25 17:59:22,2.490000,-3.234
2020-03-25 18:02:03,14.011461,2.504
2020-03-25 18:02:09,0.366000,-3.144
2020-03-25 18:02:13,9.234000,-0.811


## coincheck buyにおける売却コインの差分分析

In [15]:
cc_buy_trades = cc_trades[cc_trades["side"] == "buy"]

In [16]:
cc_buy_amount = cc_buy_trades["amount"]
min(cc_buy_amount), max(cc_buy_amount), np.mean(cc_buy_amount)

(0.00597151, 0.00600776, 0.005993331707317074)

In [17]:
from decimal import Decimal
num = 0.006 - np.mean(cc_buy_amount)
"{0:.10f}".format(Decimal(num))

'0.0000066683'

この価格差によって生じた資金移動は

In [18]:
num * len(cc_buy_trades) * np.mean(cc_ticks["ask"])

611.1558634167692

## rateの誤差比較

In [19]:
cc_rate_df = trade_df[["cc_side", "cc_expected_rate", "cc_actual_rate"]].copy()
cc_rate_df["cc_rate_diff"] = cc_rate_df["cc_expected_rate"] - cc_rate_df["cc_actual_rate"]

In [20]:
cc_rate_df

,cc_side,cc_expected_rate,cc_actual_rate,cc_rate_diff
timestamp,,,,
2020-03-25 17:03:03,sell,745000.0,743844.0,1156.0
2020-03-25 17:11:35,buy,743308.0,744023.0,-715.0
2020-03-25 17:58:47,buy,754670.0,755520.0,-850.0
2020-03-25 17:58:57,sell,755885.0,755913.0,-28.0
2020-03-25 17:59:14,buy,756999.0,758459.0,-1460.0
2020-03-25 17:59:22,sell,757661.0,756830.0,831.0
2020-03-25 18:02:03,buy,759900.0,761191.0,-1291.0
2020-03-25 18:02:09,sell,760524.0,759925.0,599.0
2020-03-25 18:02:13,sell,760524.0,759414.5,1109.5


In [21]:
cc_diff = cc_rate_df["cc_rate_diff"]
max(cc_diff), min(cc_diff), np.mean(abs(cc_diff)), np.std(cc_diff)

(2341.5, -3794.0, 895.4166666666666, 1106.636860719857)

In [22]:
lq_rate_df = trade_df[["lq_side", "lq_expected_rate", "lq_actual_rate"]].copy()
lq_rate_df["lq_rate_diff"] = lq_rate_df["lq_expected_rate"] - lq_rate_df["lq_actual_rate"]

In [23]:
lq_diff = lq_rate_df["lq_rate_diff"]
max(lq_diff), min(lq_diff), np.mean(abs(lq_diff)), np.std(lq_diff)

(1495.0, -2175.0, 317.8285894308942, 485.73299405175237)